In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gemma/keras/gemma_2b_en/1/config.json
/kaggle/input/gemma/keras/gemma_2b_en/1/tokenizer.json
/kaggle/input/gemma/keras/gemma_2b_en/1/metadata.json
/kaggle/input/gemma/keras/gemma_2b_en/1/model.weights.h5
/kaggle/input/gemma/keras/gemma_2b_en/1/assets/tokenizer/vocabulary.spm


In [2]:
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.0.5 which is incompatible.
tensorflowjs 4.16.0 requires packaging~=23.1, but you have packaging 21.3 which is incompatible.


In [3]:
import keras
import keras_nlp
import os

2024-02-21 18:34:04.129655: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-21 18:34:04.129764: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-21 18:34:04.300663: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
os.environ["KERAS_BACKEND"] = "tensorflow"  # Or "jax" or "torch".

In [5]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")

Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/1' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/1' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/gemma/keras/gemma_2b_en/1' to your Kaggle notebook...
Attaching 'tokenizer.json' from model 'keras/gemma/keras/gemma_2b_en/1' to your Kaggle notebook...
Attaching 'assets/tokenizer/vocabulary.spm' from model 'keras/gemma/keras/gemma_2b_en/1' to your Kaggle notebook...
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


In [6]:
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

In [7]:
gemma_lm.generate("What is the meaning of life?", max_length=200)

I0000 00:00:1708540563.119098      34 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1708540563.183564      34 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1708540563.496388      34 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


'What is the meaning of life?\n\nThe question is one of the most important questions in the world.\n\nIt’s the question that has been asked by philosophers, theologians, and scientists for centuries.\n\nAnd it’s the question that has been asked by people who are looking for answers to their own lives.\n\nThe meaning of life is a question that has been asked by people for centuries.\n\nThe question is one of the most important questions in the world.\n\nIt’s the question that has been asked by philosophers, theologians, and scientists for centuries.\n\nAnd it’s the question that has been asked by people who are looking for answers to their own lives.\n\nThe meaning of life is a question that has been asked by people for centuries.\n\nThe question is one of the most important questions in the world.\n\nIt’s the question that has been asked by philosophers, theologians, and scientists for centuries.\n\nAnd it’s the question that has been'

In [8]:
gemma_lm.generate("Whai is tensorflow?", max_length=200)

'Whai is tensorflow?\n\nTensorFlow is an open source software library for numerical computation using data flow graphs. It is used for machine learning and deep learning.\n\nWhat is the difference between TensorFlow and Keras?\n\nKeras is a high-level neural network library that provides an API for defining and training deep learning models. TensorFlow is a high-level neural network library that provides an API for defining and training deep learning models.\n\nWhat is the difference between TensorFlow and PyTorch?\n\nTensorFlow is a high-level neural network library that provides an API for defining and training deep learning models. PyTorch is a high-level neural network library that provides an API for defining and training deep learning models.\n\nWhat is the difference between TensorFlow and Caffe?\n\nCaffe is a deep learning framework that is based on the C++ programming language. It is used for training and deploying deep learning models. TensorFlow is a deep learning library th

In [9]:
gemma_lm.generate(
    ["What is JAX?",
     "How does JAX difference from tensorflow?"],
    max_length=200)

W0000 00:00:1708540659.419058      34 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


['What is JAX?\n\nJAX is a framework for writing and running distributed applications. It is a collection of tools that allow you to write your application in a way that is independent of the underlying infrastructure.\n\nJAX is a framework for writing and running distributed applications. It is a collection of tools that allow you to write your application in a way that is independent of the underlying infrastructure.\n\nJAX is a framework for writing and running distributed applications. It is a collection of tools that allow you to write your application in a way that is independent of the underlying infrastructure.\n\n<h2>What is JAX in Python?</h2>\n\nJAX is a Python library that provides a high-level API for writing distributed applications. It is designed to be easy to use and to provide a high level of performance.\n\nJAX is a Python library that provides a high-level API for writing distributed applications. It is designed to be easy to use and to provide a high level of perfo

In [10]:
gemma_lm.compile(sampler="top_k")   # default = "greedy"
gemma_lm.generate("What is the meaning of life?", max_length=200)

W0000 00:00:1708540733.822661      34 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1708540734.107333      34 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


'What is the meaning of life?\n\nThat’s a big question that has plagued humans for centuries, but we’ve never had an answer until now.\n\nThe meaning of life, or “what is life all about?”, is a fundamental question that has fascinated philosophers, scientists, and theologians for hundreds of years.\n\nThe search for meaning in life is one of the most important issues facing us today and it’s one that will continue to be relevant for years to come.\n\n<h2>What Is The Meaning Of Life</h2>\n\nThe meaning of life is what we humans have been asking ourselves since the beginning of time.\n\nWe have searched for answers to this question in all sorts of ways, from religion to philosophy to science, but it’s only recently that scientists have started to make some progress.\n\nThe meaning of life is what we humans have been asking ourselves since the beginning of time.\n\nThe meaning of life, or “what is the meaning of life?”\n\nis a'

In [16]:
def output(query):
    response = gemma_lm.generate(query, max_length=250)
    return display(response)

In [17]:
output("What is meaning of life?")

W0000 00:00:1708540917.255604      34 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


'What is meaning of life? Is there a purpose to life? Is there a God? These are the big questions we all ask ourselves.\n\nThe Bible answers these questions with a resounding “yes” and “yes!”\n\nThe purpose of life is to glorify God and to enjoy Him forever. This purpose is given to you and to all who believe the gospel of Christ.\n\n<blockquote><em>“I have been crucified with Christ. It is no longer I who live, but Christ lives in me. So I live in this earthly body by trusting in the Son of God, who loved me and gave himself for me.” (Galatians 2:20 NIV)</em></blockquote>\n\n<blockquote><em>“I am convinced that neither death nor life, neither angels nor demons, neither the present nor the future, nor any powers, neither height nor depth, nor anything else in all creation, will be able to separate us from the love of God that is in Christ Jesus our Lord.” (Romans 8:38,39 NIV)</em></blockquote>\n\n<blockquote><em>“For to me, to live is Christ and to die is gain. If I am to live in the f